In [ ]:
!pip install -U openai-whisper
!sudo apt install ffmpeg
!pip install fpdf
!pip install setuptools-rust

In [ ]:
import whisper
model = whisper.load_model("base")
result = model.transcribe("test.mp3")
print(result["text"])

In [41]:
def convert_seconds_to_minutes(seconds):
    minutes = seconds // 60
    seconds_remainder = seconds % 60
    return minutes + seconds_remainder / 60  # Calculate accurate minutes with decimals

In [ ]:
for a in range(len(result["segments"])):
  x=result['segments'][a]
  # Convert start and end times to minutes with decimals
  minutes_start = convert_seconds_to_minutes(x['start'])
  minutes_end = convert_seconds_to_minutes(x['end'])
  print(f"{minutes_start:.2f}:{minutes_end:.2f} {x['text']}")

In [64]:

# Create an empty string to store the output
output_text = ""

for a in range(len(result["segments"])):
    x = result["segments"][a]

    # Convert times and format output as before
    minutes_start = convert_seconds_to_minutes(x["start"])
    minutes_end = convert_seconds_to_minutes(x["end"])
    output_string = f"{minutes_start:.2f}:{minutes_end:.2f} {x['text']}\n"  # Add newline for spacing

    # Append to the output string
    output_text += output_string

# Estimate the number of pages needed based on string length and assumed character count per page
estimated_chars_per_page = 500  # Adjust as needed
estimated_pages = len(output_text) // estimated_chars_per_page + 1  # Round up


In [98]:
from fpdf import FPDF

class MyPDF(FPDF):
    def __init__(self):
        super().__init__()
        self.current_page = 1
        self.current_line = 0
        self.lines_per_page = 40  # Adjust as needed
        self.alias_nb_pages()  # Add this line
    def footer(self):
            # Position cursor at 1.5 cm from bottom:
            self.set_y(-15)
            # Setting font: helvetica italic 8
            self.set_font("helvetica", "I", 8)
            # Printing page number:
            self.cell(0, 10, f"Page {self.page_no()}/{{nb}}", align="C")
    def add_content(self, output_text):
        lines = output_text.splitlines()
        self.add_page()
        self.set_font("Arial", size=12)

        for line in lines:
            self.cell(200, 10, txt=line, ln=1)
            self.current_line += 1

            if self.get_y() > 260:  # Adjust as needed
                self.current_page += 1
                self.current_line = 0
                self.add_page()

# Example usage:
# output_text = "Your text content here..."
pdf = MyPDF()
pdf.add_content(output_text)
pdf.output("output_document.pdf")


''